In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
pairs = pd.read_csv("data/pol/pairs.csv")
len(pairs)

19999

In [3]:
data = pd.read_csv("data/pol/2-pol.csv")
data.head()

,accession,pol,query acc.ver,% identity,alignment length,mismatches,gap opens,q. start,q. end,s. start,s. end,evalue,bit score,pol_cut
0,AB098330,FFRENLAFQQGEARKFSTEQTGANSPTSGALWDGGRDILPSEAGAE...,Query_1,92.735,234,17,0,1,234,156,389,5.920000e-155,458.0,PISPIDTVPVKLKPGMDGPKVKQWPLTEEKIKALTEICIDMEKEGK...
1,AB098331,FFRENLAFQQGEARKFSTEQTGANSPTSGALWDGGRDILPSEAGTK...,Query_1,91.880,234,19,0,1,234,156,389,2.630000e-154,456.0,PISPIDTVPVKLKPGMDGPKVKQWPLTEEKVKALTEICIDMEKEGK...
2,AB098332,FFRENLAFQQGEARKFPSEQTGANSPTSRDLWNGGRDSLPSEAGAE...,Query_1,94.872,234,12,0,1,234,156,389,1.760000e-157,464.0,PISPIETVPVTLKPGMDGPKVKQWPLTEEKIKALTEICTEMEKEGK...
3,AB098333,FFRENLAFQQGEARKFPSEQTGANSPTSRDLWNGGRDSLPSEAGAE...,Query_1,94.872,234,12,0,1,234,156,389,1.760000e-157,464.0,PISPIETVPVTLKPGMDGPKVKQWPLTEEKIKALTEICTEMEKEGK...
4,AB220944,FFRENLAFQQRKAGEFSSEQTRANSPTSRKLGDGGRDNLLTEAGAE...,Query_1,93.590,234,15,0,1,234,154,387,3.240000e-156,461.0,PISPIDTIPVTLKPGMDGPKVKQWPLTEEKIKALTEICKEMEEEGK...


In [4]:
vocab = set('-')
max_seq_len = 0
for s in tqdm(data.pol):
    vocab = vocab.union(set(s))
    if len(s) > max_seq_len:
        max_seq_len = len(s)
        
vocab = sorted(vocab)
vocab_index = dict([(char, i) for i, char in enumerate(vocab)])

print(max_seq_len, ''.join(vocab))

  0%|          | 0/240393 [00:00<?, ?it/s]

1434 -ABCDEFGHIKLMNPQRSTVWXYZ


In [9]:
input_data = np.zeros(
    (len(data), max_seq_len + 1, len(vocab)), dtype = "short"
)
input_data[:,:,0] = 1
input_data.shape

(240393, 1435, 24)

In [13]:
for i, input_seq in tqdm(list(enumerate(data.pol))):
    for t, char in enumerate(input_seq):
        input_data[i, t, 0] = 0
        input_data[i, t, vocab_index[char]] = 1

  0%|          | 0/240393 [00:00<?, ?it/s]

In [19]:
pids, cids = [], []
for i, pair in tqdm(list(pairs.iterrows())):
    pid = data.index[data.accession == pair['parent']].values[0]
    cid = data.index[data.accession == pair['child']].values[0]
    pids.append(pid)
    cids.append(cid)

  0%|          | 0/19999 [00:00<?, ?it/s]

In [20]:
with open('data/pol/train.npz', 'wb') as f:
    np.savez(f, 
             parents = input_data[pids], 
             children = input_data[cids],
             vocab_index = vocab_index
            )